In [1]:
#import libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import CategoricalNB
from sklearn.multioutput import ClassifierChain
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import NearestCentroid
from sklearn.svm import NuSVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OutputCodeClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score,classification_report
import sklearn.impute as imputers
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings("ignore")
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import tools

In [2]:
from sklearn.utils import all_estimators
estimators = all_estimators(type_filter='classifier')
classification_estimators = []
i = 1
for name, class_ in estimators:
    classification_estimators.append(class_.__name__)
print(len(classification_estimators))

41


In [3]:
#load data
df = pd.read_csv('duke_vital_model_imputed.csv')
df.head()
df["PostCond"].value_counts()

1    62257
0    21063
Name: PostCond, dtype: int64

In [4]:
subj_dict = tools.create_subj_dict(df)
subj_labels = np.array([np.array(list([key, subj_dict[key][0]["PostCond"]])) for key in subj_dict.keys()])
# subject_ids = subj_conds[:, 0]
# subject_labels = subj_conds[:, 1]
skf = StratifiedKFold(n_splits = 10, shuffle=True, random_state=42)
patient_split = skf.split(subj_labels[:, 0], subj_labels[:, 1])

In [5]:
# #Splitting the dataset into the Training set and Test set
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [6]:
def filterer(pair):
    key, value = pair
    if len(value) >= 2:
        return True
    return False

In [7]:
categorical_transformer = Pipeline(
    [
        ('onehot', OneHotEncoder(handle_unknown = 'ignore'))
    ]
)
from sklearn.preprocessing import StandardScaler

numeric_transformer = Pipeline(
    [
        ('scaler', StandardScaler())
    ]
)

In [8]:
X_train = dict()
y_train = dict()
X_test = dict()
y_test = dict()
cnt = 0

from imblearn.combine import SMOTEENN
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.over_sampling import SMOTE

for train_i, test_i in patient_split:
    train_subj_dict = {patient: subj_dict[patient] for patient in subj_dict.keys() if patient in subj_labels[train_i, 0]}
    test_subj_dict = {patient: subj_dict[patient] for patient in subj_dict.keys() if patient in subj_labels[test_i, 0]}

    train_df = tools.subj_dict_to_df(train_subj_dict)
    test_df = tools.subj_dict_to_df(test_subj_dict)

    trends = tools.return_trends(train_df, [[2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13]], ["DIASBP_R", "PULSE_R", "SYSBP_R", "WEIGHT_R", "CREATININE_R", "HDL_R", "LDL_R", "TOTCHOL_R", "wbc", "gluc", "tprot", "alb", "ast", "alt", "tbil", "cr", "ptinr"])

    train_col_names = list(train_df.drop(labels=["RSUBJID", "PostCond"], axis=1).keys())
    test_col_names = list(test_df.drop(labels=["RSUBJID", "PostCond"], axis=1).keys())

    imputer = imputers.IterativeImputer(random_state=42)

    X_train_for_imputed = imputer.fit_transform(train_df.drop(labels=["RSUBJID", "PostCond"], axis=1).to_numpy())
    X_test_for_imputed = imputer.transform(test_df.drop(labels=["RSUBJID", "PostCond"], axis=1).to_numpy())

    print(np.count_nonzero(np.isnan(X_train_for_imputed)))
    print(np.count_nonzero(np.isnan(X_train_for_imputed)))
    
    imputed_train_df = pd.DataFrame(X_train_for_imputed, columns=train_col_names)
    imputed_test_df = pd.DataFrame(X_test_for_imputed, columns=test_col_names)

    imputed_train_df["RSUBJID"] = train_df["RSUBJID"]
    imputed_train_df["PostCond"] = train_df["PostCond"]
    imputed_test_df["RSUBJID"] = test_df["RSUBJID"]
    imputed_test_df["PostCond"] = test_df["PostCond"]
    print(imputed_train_df.isna().any().any())
    print(imputed_test_df.isna().any().any())

    imputed_train_subj_dict = tools.create_subj_dict(imputed_train_df)
    imputed_test_subj_dict = tools.create_subj_dict(imputed_test_df)

    print(f'imputed_train_subj_dict: {tools.check_subj_dict_dim_err(imputed_train_subj_dict)}')

    filtered_imputed_train_subj_dict = dict(filter(filterer, imputed_train_subj_dict.items()))
    filtered_imputed_test_subj_dict = dict(filter(filterer, imputed_test_subj_dict.items()))

    print(f'filtered_imputed_train_subj_dict: {tools.check_subj_dict_dim_err(filtered_imputed_train_subj_dict)}')

    normalized_train_dict = tools.dist_from_age_mean(filtered_imputed_train_subj_dict, trends)
    normalized_test_dict = tools.dist_from_age_mean(filtered_imputed_test_subj_dict, trends)

    print(f'normalized_train_dict: {tools.check_subj_dict_dim_err(normalized_train_dict)}')
    print(tools.subj_dict_nan(normalized_train_dict))

    # def check_nans_row_lst(pair):
    #     key, lst = pair
    #     return any(list(map(lambda x: any(list(map(lambda n: np.isnan(n[1]), x.items()))), lst)))

    linearized_train_dict = tools.dict_to_linfit(normalized_train_dict, trend_cols=["DIASBP_R", "PULSE_R", "SYSBP_R", "WEIGHT_R", "CREATININE_R", "HDL_R", "LDL_R", "TOTCHOL_R", "wbc", "gluc", "tprot", "alb", "ast", "alt", "tbil", "cr", "ptinr"])
    linearized_test_dict = tools.dict_to_linfit(normalized_test_dict, trend_cols=["DIASBP_R", "PULSE_R", "SYSBP_R", "WEIGHT_R", "CREATININE_R", "HDL_R", "LDL_R", "TOTCHOL_R", "wbc", "gluc", "tprot", "alb", "ast", "alt", "tbil", "cr", "ptinr"])

    print(tools.check_lin_dict_dim_err(linearized_train_dict))
    print(tools.lin_dict_nan(linearized_train_dict))

    linearized_train_df = tools.lin_dict_to_df(linearized_train_dict)
    linearized_test_df = tools.lin_dict_to_df(linearized_test_dict)

    print(linearized_train_df.isna().any().any())

    X_train_i_df = linearized_train_df.drop(labels=["RSUBJID", "PostCond"], axis=1)
    X_test_i_df = linearized_test_df.drop(labels=["RSUBJID", "PostCond"], axis=1)
    y_train_i = linearized_train_df["PostCond"]
    y_test_i = linearized_test_df["PostCond"]

    categorical_columns = ["RACE_G", "GENDER", "HXCOPD", "HXDIAB", "HXHTN", "HXHYL", "HXSMOKE"]
    numeric_columns = [col for col in X_train_i_df.columns if col not in categorical_columns]

    preprocessor = ColumnTransformer(
        [
            ('categoricals', categorical_transformer, categorical_columns),
            ('numericals', numeric_transformer, numeric_columns)
        ],
        remainder = 'passthrough',
        sparse_threshold=0
    )

    print(X_train_i_df.isna().any().any())

    X_train_i = preprocessor.fit_transform(X_train_i_df)
    X_test_i = preprocessor.transform(X_test_i_df)

    print(np.count_nonzero(np.isnan(X_train_i)))
    print(np.count_nonzero(np.isnan(y_train_i)))



    # X_train_i = X[train_i]
    # X_test_i = X[test_i]
    # y_train_i = y[train_i]
    # y_test_i = y[test_i]

    resampler = SMOTE()
    
    print(f'train len before resample: {len(X_train_i)}')
    X_train_i, y_train_i = resampler.fit_resample(X_train_i, y_train_i)
    print(f'train len after resample: {len(X_train_i)}')
    print(f'Num of positive train: {np.sum(y_train_i)}')
    print(f'Num of negative train: {len(y_train_i) - np.sum(y_train_i)}')
    
    X_train[cnt] = X_train_i
    X_test[cnt] = X_test_i
    y_train[cnt] = y_train_i
    y_test[cnt] = y_test_i

    cnt += 1


0
0
False
False
imputed_train_subj_dict: False
filtered_imputed_train_subj_dict: False
normalized_train_dict: False
False
False
False
False
False
0
0
train len before resample: 15312
train len after resample: 23396
Num of positive train: 11698.0
Num of negative train: 11698.0
0
0
False
False
imputed_train_subj_dict: False
filtered_imputed_train_subj_dict: False
normalized_train_dict: False
False
False
False
False
False
0
0
train len before resample: 15383
train len after resample: 23506
Num of positive train: 11753.0
Num of negative train: 11753.0
0
0
False
False
imputed_train_subj_dict: False
filtered_imputed_train_subj_dict: False
normalized_train_dict: False
False
False
False
False
False
0
0
train len before resample: 15340
train len after resample: 23418
Num of positive train: 11709.0
Num of negative train: 11709.0
0
0
False
False
imputed_train_subj_dict: False
filtered_imputed_train_subj_dict: False
normalized_train_dict: False
False
False
False
False
False
0
0
train len before re

In [9]:
# #Scaling the Dataset
# from sklearn.preprocessing import StandardScaler
# sc1 = StandardScaler()
# sc2 = StandardScaler()
# imputer = imputers.IterativeImputer(random_state=42)
# # X_train = sc1.fit_transform(X_train)
# X_train = imputer.fit_transform(X_train)
# X_train = sc2.fit_transform(X_train)
# # X_test = sc1.transform(X_test)
# X_test = imputer.transform(X_test)
# X_test = sc2.transform(X_test)

In [10]:
models=[AdaBoostClassifier,BaggingClassifier,BernoulliNB,CalibratedClassifierCV,DecisionTreeClassifier,DummyClassifier,
        ExtraTreeClassifier,ExtraTreesClassifier,GaussianNB,GradientBoostingClassifier,HistGradientBoostingClassifier,KNeighborsClassifier,
        LinearDiscriminantAnalysis,LinearSVC,LogisticRegression,LogisticRegressionCV,MLPClassifier,
        NearestCentroid,PassiveAggressiveClassifier,Perceptron,QuadraticDiscriminantAnalysis,
        RandomForestClassifier,RidgeClassifier,RidgeClassifierCV,SGDClassifier,SVC] 

cross_val_models = dict()
for i in range(10):
        cross_val_models[i] = list()
        for model in models:
                cross_val_models[i].append(model)
            
#CategoricalNB
#ClassifierChain
#ComplementNB
#GaussianProcessClassifier
#LabelPropagation
#LabelSpreading
#MultinomialNB
#NuSVC
#OneVsOneClassifier
#OneVsRestClassifier,
#OutputCodeClassifier
#RadiusNeighborsClassifier
#StackingClassifier
#VotingClassifier

In [11]:
#Fitting the Model
from statistics import mean
from sklearn.metrics import balanced_accuracy_score, recall_score, precision_score, f1_score, accuracy_score
model_accuracies =[]
model_balanced_accuracies = []
model_recalls = []
model_precisions = []
model_f1 = []

for i in range(len(models)):
    accuracies = list()
    balanced_accuracies = list()
    recalls = list()
    precisions = list()
    f1 = list()
    for k in X_train.keys():
        model = cross_val_models[k][i]()
        model.fit(X_train[k], y_train[k])
        y_pred= model.predict(X_test[k])
        accuracies.append(round(accuracy_score(y_test[k], y_pred),4))
        balanced_accuracies.append(round(balanced_accuracy_score(y_test[k], y_pred),4))
        recalls.append(round(recall_score(y_test[k], y_pred), 4))
        precisions.append(round(precision_score(y_test[k], y_pred), 4))
        f1.append(round(f1_score(y_test[k], y_pred), 4))

    model_accuracies.append(mean(accuracies))
    model_balanced_accuracies.append(mean(balanced_accuracies))
    model_recalls.append(mean(recalls))
    model_precisions.append(mean(precisions))
    model_f1.append(mean(f1))

    print (f'model : {models[i]} and  accuracy score is : {mean(accuracies)}, balanced accuracy score: {mean(balanced_accuracies)}, recall: {mean(recalls)}, precision: {mean(precisions)}, f1: {mean(f1)}')
    
result_df1 = pd.DataFrame({'Accuracy':model_accuracies, 'Balanced Accuracy': model_balanced_accuracies, 'Recall':model_recalls, 'Precision':model_precisions, 'f1':model_f1}, index=models)
result_df1

model : <class 'sklearn.ensemble._weight_boosting.AdaBoostClassifier'> and  accuracy score is : 0.66115, balanced accuracy score: 0.60354, recall: 0.71265, precision: 0.82034, f1: 0.76257
model : <class 'sklearn.ensemble._bagging.BaggingClassifier'> and  accuracy score is : 0.66475, balanced accuracy score: 0.6107400000000001, recall: 0.71304, precision: 0.82454, f1: 0.76457
model : <class 'sklearn.naive_bayes.BernoulliNB'> and  accuracy score is : 0.54026, balanced accuracy score: 0.54915, recall: 0.53236, precision: 0.79896, f1: 0.63859
model : <class 'sklearn.calibration.CalibratedClassifierCV'> and  accuracy score is : 0.30037, balanced accuracy score: 0.47899, recall: 0.14099, precision: 0.71486, f1: 0.23538
model : <class 'sklearn.tree._classes.DecisionTreeClassifier'> and  accuracy score is : 0.63609, balanced accuracy score: 0.5758300000000001, recall: 0.68991, precision: 0.8059, f1: 0.74338
model : <class 'sklearn.dummy.DummyClassifier'> and  accuracy score is : 0.235860000000

,Accuracy,Balanced Accuracy,Recall,Precision,f1
<class 'sklearn.ensemble._weight_boosting.AdaBoostClassifier'>,0.66115,0.60354,0.71265,0.82034,0.76257
<class 'sklearn.ensemble._bagging.BaggingClassifier'>,0.66475,0.61074,0.71304,0.82454,0.76457
<class 'sklearn.naive_bayes.BernoulliNB'>,0.54026,0.54915,0.53236,0.79896,0.63859
<class 'sklearn.calibration.CalibratedClassifierCV'>,0.30037,0.47899,0.14099,0.71486,0.23538
<class 'sklearn.tree._classes.DecisionTreeClassifier'>,0.63609,0.57583,0.68991,0.80590,0.74338
<class 'sklearn.dummy.DummyClassifier'>,0.23586,0.50000,0.00000,0.00000,0.00000
<class 'sklearn.tree._classes.ExtraTreeClassifier'>,0.61755,0.53655,0.68980,0.78364,0.73364
<class 'sklearn.ensemble._forest.ExtraTreesClassifier'>,0.76695,0.57875,0.93491,0.79578,0.85973
<class 'sklearn.naive_bayes.GaussianNB'>,0.24701,0.50329,0.01818,0.83278,0.03555
<class 'sklearn.ensemble._gb.GradientBoostingClassifier'>,0.78704,0.62046,0.93584,0.81354,0.87035


In [1]:
print(X_train[0].shape())

NameError: name 'X_train' is not defined